In [1]:
import pandas as pd
from os import path
from datetime import date, timedelta

In [2]:
duration_start = date.today() - timedelta(days=7)

districts = pd.read_table(path.join('..', 'raw_data', 'shanghai_districts_detailed.tsv')).astype({'计数':'int64'})

districts = districts[districts['日期'] >= duration_start.strftime("%Y-%m-%d")]
districts = districts[districts['来源口径'] != "转诊"]

In [3]:
districts

,日期,区县,诊断,来源口径,计数
3360,2022-05-07,浦东,确诊,闭环,22
3361,2022-05-07,浦东,确诊,社会面,0
3363,2022-05-07,浦东,无症状,闭环,641
3364,2022-05-07,浦东,无症状,社会面,6
3365,2022-05-07,黄浦,确诊,闭环,7
...,...,...,...,...,...
3914,2022-05-13,奉贤,无症状,社会面,0
3915,2022-05-13,崇明,确诊,闭环,0
3916,2022-05-13,崇明,确诊,社会面,0
3918,2022-05-13,崇明,无症状,闭环,2


In [4]:
districts = districts.groupby(['区县','日期','来源口径'])['计数'].agg(计数='sum')

In [5]:
districts

计数
区县 日期         来源口径     
嘉定 2022-05-07 社会面     0
              闭环    108
   2022-05-08 社会面     0
              闭环    110
   2022-05-09 社会面     1
...                 ...
黄浦 2022-05-11 闭环    228
   2022-05-12 社会面     0
              闭环    274
   2022-05-13 社会面     0
              闭环    221

[224 rows x 1 columns]

In [6]:
daily_subtotal = districts.rename(columns={'计数':'单日计'}).sum(level=[0,1]).assign(来源口径='单日计').set_index('来源口径', append=True)

In [7]:
districts

计数
区县 日期         来源口径     
嘉定 2022-05-07 社会面     0
              闭环    108
   2022-05-08 社会面     0
              闭环    110
   2022-05-09 社会面     1
...                 ...
黄浦 2022-05-11 闭环    228
   2022-05-12 社会面     0
              闭环    274
   2022-05-13 社会面     0
              闭环    221

[224 rows x 1 columns]

In [8]:
districts = pd.concat([districts,daily_subtotal], axis=1, join='outer',levels=[0,1,2])

In [9]:
# final_districts[]
districts = districts.fillna(0)
districts = districts.astype('int32')
districts['单项'] = districts['计数'] + districts['单日计']
districts = districts.drop(columns=['计数', '单日计'])

In [10]:
yesterday = date.today() - timedelta(days=1)

export_districts = districts.unstack().unstack()
export_districts = export_districts.sort_values([('单项','单日计', yesterday.strftime("%Y-%m-%d"))], ascending=True)

In [11]:
export_districts

单项                                                         \
来源口径        单日计                                                          
日期   2022-05-07 2022-05-08 2022-05-09 2022-05-10 2022-05-11 2022-05-12   
区县                                                                       
奉贤           11          2          5          0          5          1   
崇明           51         13          6          8          4          6   
金山            0          2          1          0          3          1   
松江           29         30         20         15         13         16   
青浦           32         48         54         16         35         19   
普陀           54         47         49         25         27         28   
长宁           62         72         52         14         39         59   
嘉定          108        110        132         15         73         67   
徐汇          295        291        119         62         65         91   
虹口          262        279        250        100         97         99   
闵行          221        220        319         72         88        129   
宝山          325        315        144        132        125        162   
静安          532        598        378        196        161        183   
浦东          669        677        582        250        249        273   
黄浦          582        449        383        248        228        274   
杨浦          607        564        364        136        131        521   

                                                  ...                        \
来源口径                   社会面                        ...                         
日期   2022-05-13 2022-05-07 2022-05-08 2022-05-09  ... 2022-05-11 2022-05-12   
区县                                                ...                         
奉贤            0          2          0          0  ...          1          0   
崇明            3          0          0          0  ...          0          0   
金山            7          0          0          0  ...          0          0   
松江            9          0          5          0  ...          0          0   
青浦           15          0          0          1  ...          0          0   
普陀           25          0          0          0  ...          0          0   
长宁           37          0          0          0  ...          0          0   
嘉定           40          0          0          1  ...          0          0   
徐汇           83          1          1          0  ...          1          0   
虹口           88          1          0          0  ...          0          0   
闵行          121          1          0          0  ...          0          0   
宝山          152          0          0          0  ...          0          0   
静安          162          0          1          1  ...          0          0   
浦东          179          6          2          1  ...          0          4   
黄浦          221          0          0          1  ...          0          0   
杨浦          399          0          0          0  ...          0          0   

                                                                        \
来源口径                    闭环                                               
日期   2022-05-13 2022-05-07 2022-05-08 2022-05-09 2022-05-10 2022-05-11   
区县                                                                       
奉贤            0          9          2          5          0          4   
崇明            1         51         13          6          8          4   
金山            0          0          2          1          0          3   
松江            0         29         25         20         15         13   
青浦            0         32         48         53         16         35   
普陀            0         54         47         49         25         27   
长宁            0         62         72         52         14         39   
嘉定            0        108        110        131         15         73   
徐汇            0        294        290        119         62    

In [12]:
basepath= path.join('..', '..', 'COVID_JS_INTERACTIVE', 'covid_5cities_jsdist', 'dailyCovid')

with open(path.join(basepath, 'districtwise_within_quarantine.csv'), 'w+', encoding='utf-8', newline='') as chart_dataset:
    export_districts['单项']['闭环'].to_csv(chart_dataset)

with open(path.join(basepath, 'districtwise_outside_quarantine.csv'), 'w+', encoding='utf-8', newline='') as chart_dataset:
    export_districts['单项']['社会面'].to_csv(chart_dataset)